# Import modules

In [1]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

In [164]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_data = """query data ($nationality: String!, $job: String!)
            {movieList(nationality: $nationality, job: $job)
            {
                movieTitle 
                movieRevenue
                }}"""
query_introspect = """{
  __schema {
    types {
      name
    }
  }
}"""
query_metadata = """query metadata {nationalityList(job:"Director"), jobList(nationality:"Arabic")}"""
query_playground = """{"operationName":"IntrospectionQuery","variables":{},"query":"query IntrospectionQuery {\n  __schema {\n    queryType {\n      name\n    }\n    mutationType {\n      name\n    }\n    subscriptionType {\n      name\n    }\n    types {\n      ...FullType\n    }\n    directives {\n      name\n      description\n      locations\n      args {\n        ...InputValue\n      }\n    }\n  }\n}\n\nfragment FullType on __Type {\n  kind\n  name\n  description\n  fields(includeDeprecated: true) {\n    name\n    description\n    args {\n      ...InputValue\n    }\n    type {\n      ...TypeRef\n    }\n    isDeprecated\n    deprecationReason\n  }\n  inputFields {\n    ...InputValue\n  }\n  interfaces {\n    ...TypeRef\n  }\n  enumValues(includeDeprecated: true) {\n    name\n    description\n    isDeprecated\n    deprecationReason\n  }\n  possibleTypes {\n    ...TypeRef\n  }\n}\n\nfragment InputValue on __InputValue {\n  name\n  description\n  type {\n    ...TypeRef\n  }\n  defaultValue\n}\n\nfragment TypeRef on __Type {\n  kind\n  name\n  ofType {\n    kind\n    name\n    ofType {\n      kind\n      name\n      ofType {\n        kind\n        name\n        ofType {\n          kind\n          name\n          ofType {\n            kind\n            name\n            ofType {\n              kind\n              name\n              ofType {\n                kind\n                name\n              }\n            }\n          }\n        }\n      }\n    }\n  }\n}\n"}"""
query_data = json.dumps({'variables': {'nationality': 'English', 'job': 'Director'}, 'query': query_data})
query_metadata = json.dumps({'query': query_metadata})

# Test handler functions

In [161]:
import handler 
importlib.reload(handler);

In [167]:
%%timeit
out = handler.handle(query_data)

188 ms ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [166]:
out

'{"data": {"movieList": [{"movieTitle": "The Lord of the Rings: The Return of the King", "movieRevenue": 1118}, {"movieTitle": "The Hobbit: An Unexpected Journey", "movieRevenue": 1021}, {"movieTitle": "The Hobbit: The Desolation of Smaug", "movieRevenue": 958}, {"movieTitle": "The Hobbit: The Battle of the Five Armies", "movieRevenue": 956}, {"movieTitle": "The Lord of the Rings: The Two Towers", "movieRevenue": 926}, {"movieTitle": "The Lord of the Rings: The Fellowship of the Ring", "movieRevenue": 871}, {"movieTitle": "King Kong", "movieRevenue": 550}, {"movieTitle": "The Lovely Bones", "movieRevenue": 93}, {"movieTitle": "The Frighteners", "movieRevenue": 29}, {"movieTitle": "Heavenly Creatures", "movieRevenue": 3}, {"movieTitle": "Harry Potter and the Goblet of Fire", "movieRevenue": 895}, {"movieTitle": "Prince of Persia: The Sands of Time", "movieRevenue": 335}, {"movieTitle": "Four Weddings and a Funeral", "movieRevenue": 254}, {"movieTitle": "Donnie Brasco", "movieRevenue": 4

In [160]:
with open(str(bundle_root / 'functions/index_by_nationality/schema.json'), 'w') as fp:
    json.dump(handler.schema.introspect(), fp)

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [163]:
function_url = 'https://labs.cloud.contiamo.com/b0a002bf-d166-5ccc-a552-ebe89882cb90/v1/function/be852eaa-c701-4f73-8ed7-aceb6dd4aeb8/index_by_nationality'

In [167]:
%%time
deployed_output = requests.post(function_url,
                                data=input_query_data).json()

CPU times: user 20.3 ms, sys: 30 µs, total: 20.4 ms
Wall time: 233 ms


In [168]:
deployed_output

{'data': {'movieList': [{'title': 'The Lord of the Rings: The Return of the King',
    'revenue': 1118888979},
   {'title': 'The Hobbit: An Unexpected Journey', 'revenue': 1021103568},
   {'title': 'The Hobbit: The Desolation of Smaug', 'revenue': 958400000}]},
 'errors': None}

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True